In [71]:
import datetime as dt
from dataclasses import dataclass
from typing import TypedDict
from enum import Enum

@dataclass
class ChickTimerStatus():
    DaysSinceStatusChanged: int
    DaysSinceHatch: int
    DaysSinceDesertionTriggered: int
    TimeOfEmergence: int
    WeeksOfBatteryLeft: int
    ActivityYesterday: int
    ActivityTwoDaysAgo: int
    MeanActivity: int

class ChickTimerMode(Enum):
    NotIncubating = 30
    Incubating = 48
    Mortality = 80

class ChickTimer():
    status : ChickTimerStatus
    channel : int

    def __init__(self):
        self.status = ChickTimerStatus(
            DaysSinceStatusChanged = 0,
            DaysSinceHatch = 1,
            DaysSinceDesertionTriggered = 2,
            TimeOfEmergence = 3,
            WeeksOfBatteryLeft = 10,
            ActivityYesterday = 0,
            ActivityTwoDaysAgo = 0,
            MeanActivity = 0
        )
        self.channel = 0

    def __str__(self):
        return f'ChickTimer:\n\t{self.status}\n\tChannel: {self.channel}'

In [72]:
ct = ChickTimer()
print(ct)
ct.status.DaysSinceHatch = 3
print(ct)

ChickTimer:
	ChickTimerStatus(DaysSinceStatusChanged=0, DaysSinceHatch=1, DaysSinceDesertionTriggered=2, TimeOfEmergence=3, WeeksOfBatteryLeft=10, ActivityYesterday=0, ActivityTwoDaysAgo=0, MeanActivity=0)
	Channel: 0
ChickTimer:
	ChickTimerStatus(DaysSinceStatusChanged=0, DaysSinceHatch=3, DaysSinceDesertionTriggered=2, TimeOfEmergence=3, WeeksOfBatteryLeft=10, ActivityYesterday=0, ActivityTwoDaysAgo=0, MeanActivity=0)
	Channel: 0
